In [1]:
import numpy as np

import torch

from transformers import (AutoTokenizer, 
                          AutoModelForQuestionAnswering, 
                          BertTokenizer, 
                          BertModel)

from scipy.spatial.distance import cosine

from datasets import load_dataset

from tqdm import tqdm

c:\Users\Sanya\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Настройка

In [2]:
# Датасет
DATASET_NAME = "sberquad"

# Reader
READER_MODEL_NAME = 'AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru'

# Retriever
RETRIEVER_MODEL_NAME = 'DeepPavlov/rubert-base-cased'

### Устройство для обучения

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Retriever

In [4]:
retriever_tokenizer = BertTokenizer.from_pretrained(RETRIEVER_MODEL_NAME, do_lower_case = True)
retriever_model = BertModel.from_pretrained(RETRIEVER_MODEL_NAME, output_hidden_states = True).to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def vectors_from_texts(text):

    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = retriever_tokenizer.tokenize(marked_text)

    if len(tokenized_text) > 512:
        tokenized_text = retriever_tokenizer.tokenize(marked_text)[:512]

    # Map the token strings to their vocabulary indeces.
    indexed_tokens = retriever_tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)

    with torch.no_grad():
        outputs = retriever_model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.stack(hidden_states, dim = 0)
    token_embeddings = torch.squeeze(token_embeddings, dim = 1)
    token_embeddings = token_embeddings.permute(1, 0, 2)
    token_vecs_sum = []

    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim = 0)
        token_vecs_sum.append(sum_vec)

        token_vecs = hidden_states[-2][0]

    # Calculate the average of all n token vectors.
    sentence_embedding = torch.mean(token_vecs, dim = 0)
    return sentence_embedding.cpu().numpy()

In [6]:
def retriever(context_vectors, question):

    question_vector = vectors_from_texts(question)
    min_cosine_id = np.argmin([cosine(vector, question_vector) for vector in context_vectors])

    return min_cosine_id

# Reader

In [7]:
reader_tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
reader_model = AutoModelForQuestionAnswering.from_pretrained(READER_MODEL_NAME).to(device)

In [8]:
def reader(text, question):

    encoding = reader_tokenizer.encode_plus(text = question, text_pair = text)

    inputs = encoding['input_ids']
    tokens = reader_tokenizer.convert_ids_to_tokens(inputs)
    
    output = reader_model(input_ids = torch.tensor([inputs], device = device))

    start_index = torch.argmax(output[0])
    end_index = torch.argmax(output[1])

    answer = ' '.join(tokens[start_index : end_index + 1])
    answer = answer.replace('_', ' ')
    answer = answer.replace('▁', '')

    return answer

# Основная функция

In [9]:
dataset = load_dataset(DATASET_NAME)
np.unique(dataset['train']['context'])[:10]

array([' Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].',
       ' Freewheelin’ Bob Dylan произвёл большое впечатление не только на любителей фолк-музыки и не только в США. Кумиры британской публики The Beatles, по свидетельств

In [10]:
def reader_retriever(questions):

    dataset = load_dataset(DATASET_NAME)
    texts = np.unique(dataset['train']['context'])

    context_vectors = [vectors_from_texts(text) for text in tqdm(texts, desc = 'Векторизация текстов')]

    for question in questions:
        text_id = retriever(context_vectors, question)
        answer = reader(texts[text_id], question)

        print(f'Вопрос: {question.strip()}')
        print(f'Текст c ответом: {texts[text_id].strip()}')
        print(f'Ответ: {answer.strip()}\n')

# Проверка работы

In [11]:
reader_retriever(np.random.choice(dataset['test']['question'], size = 10))

Векторизация текстов: 100%|██████████| 9078/9078 [01:28<00:00, 102.86it/s]


Вопрос: С кем из посиленцев сотрудничали в Онтарио ирокезы?
Текст c ответом: Пьер Готье де Варенн, сьёр де Ла-Верандри посетил долину Ред-Ривер в 1730 году, чтобы помочь открыть область для французской разведки и торговли[37]. После того как французские путешественники исследовали этот район, Монреальская Северо-Западная компания начала торговать с метисами. И Монреальская Северо-Западная компания, и Компания Гудзонова залива строили форты для торговли мехами; две компании конкурировали в южной части провинции Манитоба, что иногда приводило к вспышкам насилия, — до тех пор, пока они не объединились в 1821 году[35]. Великобритания закрепила за собой эти территории в 1763 году в результате победы над Францией в Семилетней войне (также известной как франко-индейская война, 1754—1763).
Ответ: Мон ре а льская Север о - Запад ная компания

Вопрос: Кто был наследницей Вольтера?
Текст c ответом: К писателям, чьё творчество оказало влияние на Лавкрафта, что можно увидеть по его рецензиям в эссе